<img src="./Img/Label_02.png">

# Зачет

28 мая 2021 года <br>
ПИ19-3, ПИ19-4 - 3 подгруппа<br>

5 июня 2021 года <br>
ПИ19-2, ПИ19-3, ПИ19-4 - 2 подгруппа<br>

4 июня 2021 года <br>
ПИ19-4, ПИ19-5 - 4 подгруппа

### Деменчук Георгий, ПИ19-4

In [50]:
#14

"""
ORM
Напишите запрос, выполняющий вывод имен и фамилий студентов, 
получивших хотя бы одну отличную оценку.
"""

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, DateTime, ForeignKey, Integer, String, Text
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base


Base = declarative_base()
metadata = Base.metadata


class City(Base):
    __tablename__ = 'city'

    city_id = Column(Integer, primary_key=True)
    name = Column(Text)


class Subject(Base):
    __tablename__ = 'subject'

    subj_id = Column(Integer, primary_key=True)
    subj_name = Column(String(64), nullable=False, index=True)
    hour = Column(Integer, nullable=False, index=True)
    semester = Column(Integer, nullable=False)


class University(Base):
    __tablename__ = 'university'

    univ_id = Column(Integer, primary_key=True)
    univ_name = Column(String(255))
    rating = Column(Integer)
    city = Column(Integer)


class Lecturer(Base):
    __tablename__ = 'lecturer'

    lecturer_id = Column(Integer, primary_key=True)
    surname = Column(String(30), nullable=False)
    name = Column(String(30), nullable=False)
    city = Column(String(30))
    univ_id = Column(ForeignKey('university.univ_id'), nullable=False)

    univ = relationship('University')


class Student(Base):
    __tablename__ = 'student'

    student_id = Column(Integer, primary_key=True)
    surname = Column(String(255), nullable=False)
    name = Column(String(255), nullable=False)
    stipend = Column(Integer, nullable=False)
    kurs = Column(Integer, nullable=False)
    city = Column(String(300))
    birthday = Column(DateTime)
    univ_id = Column(ForeignKey('university.univ_id'), nullable=False)

    univ = relationship('University')


class ExamMark(Base):
    __tablename__ = 'exam_marks'

    exam_id = Column(Integer, primary_key=True)
    student_id = Column(ForeignKey('student.student_id'))
    subj_id = Column(ForeignKey('subject.subj_id'))
    mark = Column(Integer)
    exam_date = Column(DateTime, nullable=False)

    student = relationship('Student')
    subj = relationship('Subject')


class SubjLect(Base):
    __tablename__ = 'subj_lect'

    subj_lect_id = Column(Integer, primary_key=True)
    lecturer_id = Column(ForeignKey('lecturer.lecturer_id'))
    subj_id = Column(ForeignKey('subject.subj_id'))

    lecturer = relationship('Lecturer')
    subj = relationship('Subject')

engine = create_engine('sqlite:///Students_2021.sqlite') # 2b

Session = sessionmaker(bind=engine) # 3

session = Session() # 4



q = session.query(Student).join(ExamMark)
q = q.filter(ExamMark.mark==5)
for item in q.all():
    print(item.student_id, item.surname, item.name)

56 Сергеева Елизавета
32 Котов Павел
100 Никитин Самуил
1 Иванов Иван
76 Мельникова Дарья
55 Белкин Вадим
12 Зайцева Ольга


In [51]:
#16

"""
Core
Напишите запрос, который выполняет выборку фамилий ВСЕХ студентов, 
с указанием для студентов, сдававших экзамены, идентификаторов сданных ими предметов обучения.
"""
from sqlalchemy import (select, create_engine, MetaData, Table, Numeric, String, 
                        insert, update, delete, cast)
from datetime import datetime
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

engine=create_engine('sqlite:///Students_2021.sqlite')
metadata=MetaData()
connection=engine.connect()

city=Table('city', metadata, autoload=True, autoload_with=engine)
subject=Table('subject', metadata, autoload=True, autoload_with=engine)
university=Table('university', metadata, autoload=True, autoload_with=engine)
lecturer=Table('lecturer', metadata, autoload=True, autoload_with=engine)
student=Table('student', metadata, autoload=True, autoload_with=engine)
exam_marks=Table('exam_marks', metadata, autoload=True, autoload_with=engine)
subj_lect=Table('subj_lect', metadata, autoload=True, autoload_with=engine)

joined_tables = student.join(exam_marks, student.c.student_id==exam_marks.c.student_id, isouter=True)
s = select([
    student.c.student_id,
    student.c.name,
     student.c.surname,
    exam_marks.c.mark,
    exam_marks.c.subj_id,
    ]
    ).select_from(joined_tables)

print(str(s))
rp = connection.execute(s)
data_dict = {}

#Из полученных данных формируем словарь с оценками по предметам
for item in rp.fetchall():    
    if item.student_id not in data_dict:
        data_dict[item.student_id] = {"marks" : [], "name": item.name , "surname": item.surname}
    elif item.mark != None:
        data_dict[item.student_id]["marks"].append(item.mark)

#Просто для красивого вывода
for k,v in data_dict.items():
    if len(v["marks"]) == 0:
        data_dict[k]["marks"] = "не сдавал"
    else:
        data_dict[k]["marks"] = "предметы: " + ", ".join(map(str,v["marks"]))
        
#Вывод
for k, v in data_dict.items():
    print(f"{k} {v['surname']} {v['name']}, {v['marks']}")

SELECT student.student_id, student.name, student.surname, exam_marks.mark, exam_marks.subj_id 
FROM student LEFT OUTER JOIN exam_marks ON student.student_id = exam_marks.student_id
1 Иванов Иван, предметы: 4, 3
3 Петров Петр, не сдавал
6 Сидоров Вадим, не сдавал
10 Кузнецов Борис, не сдавал
12 Зайцева Ольга, предметы: 5, 4
32 Котов Павел, предметы: 4, 3, 5, 4
55 Белкин Вадим, предметы: 4, 3, 3, 3
56 Сергеева Елизавета, предметы: 5, 3, 5, 3, 2
62 Семенова Вероника, предметы: 4, 4
63 Медведева Ирина, не сдавал
64 Афанасьева Оксана, не сдавал
71 Синицин Кондрат, предметы: 4
76 Мельникова Дарья, предметы: 3, 3, 5, 3
100 Никитин Самуил, предметы: 4, 3, 4, 3, 5
101 Калашников Евгений, предметы: 3
102 Петров Касьян, не сдавал
103 Шоу Бернар, не сдавал
104 Доу Джо, не сдавал
105 Юнг Карл, не сдавал


## Задание

Решите задачи, номера которых указаны напротив вашего имени в таблице. Используйте методы SQLAlchemy ORM для задачи в столбце ORM и SQLAlchemy Core для задачи в столбце Core.